<a href="https://colab.research.google.com/github/Knight7561/hf-smol-course/blob/learn-course/1_instruction_tuning/student_examples/Knight7561/python-code-DPO-fine-tune/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [2]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

# from huggingface_hub import login
# login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [15]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
# ds = load_dataset(path="CoolSpring/morse-code-mistral-dpo")
ds = load_dataset(path="quangduc1112001/python-code-DPO-fine-tune")



README.md:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/870k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/99.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [16]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2_python_coder"
finetune_tags = ["smol-course-learn", "module_1","python_coder"]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [17]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [18]:
# Test the fine-tuned model on the same prompt

def test_finetuned_model(prompt):
  # Let's test the base model before training

  # Format with template
  messages = [{"role": "user", "content": prompt}]
  formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

  # Generate response
  inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

  # TODO: use the fine-tuned to model generate a response, just like with the base example.
  output=model.generate(**inputs,max_new_tokens=100)
  deocded_output=tokenizer.decode(output[0], skip_special_tokens=True)
  return deocded_output

In [19]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1800
    })
    validation: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 200
    })
})

In [20]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)
def add_chat_template(row):
  messages=[
        {"role": "user", "content": row['prompt']},
        {"role": "assistant", "content": row['chosen']}
    ]
  row['tokenized_message']=tokenizer.apply_chat_template(messages,tokenize=False)
  return row

ds=ds.map(add_chat_template)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [22]:
ds['train'][3]['tokenized_message']

'<|im_start|>user\nCompose a line of Python code to insert an element into a given list.<|im_end|>\n<|im_start|>assistant\nlist1 = [1, 3, 7, 8]\nelement = 5\n\nlist1.insert(2, element)\nprint("New list:", list1)<|im_end|>\n'

In [24]:
test_finetuned_model(ds['train'][4]['prompt'])

'user\nOptimize the following Python code for faster runtime performance\n\nimport sys\nimport os\nimport time\nimport random\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport matplotlib.animation as animation\nimport matplotlib.dates as mdates\nimport matplotlib.colors as mcolors\nimport matplotlib.animation as animation\nimport matplotlib.animation_examples as ani\nimport matplotlib.animation as animation\nimport matplotlib.animation_examples as ani\nimport matplotlib.animation as animation\nimport matplotlib.animation'

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [26]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=200,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    report_to=None,
    max_seq_length=1024,
    dataset_text_field="tokenized_message"
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    processing_class=tokenizer,
    eval_dataset=ds["validation"]
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [27]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
200,1.045500,0.957537
400,0.925800,0.939671
600,0.740100,0.942887
800,0.731300,0.935059
1000,0.597300,0.942642


The above training was stopped because there was no apperciable improvement observed in the epochs.

In [46]:
# Set our name for the finetune to be saved &/ uploaded to
trainer.push_to_hub(tags=finetune_tags)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/knight7561/SmolLM2-morse-code/commit/7096164ef0de64108e1e4f76a7a3ad01692fe065', commit_message='End of training', commit_description='', oid='7096164ef0de64108e1e4f76a7a3ad01692fe065', pr_url=None, repo_url=RepoUrl('https://huggingface.co/knight7561/SmolLM2-morse-code', endpoint='https://huggingface.co', repo_type='model', repo_id='knight7561/SmolLM2-morse-code'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [32]:
model=model.from_pretrained('/content/sft_output/checkpoint-1000').to(device)

In [38]:
def test_on_dataset(ds_row):
  print("####################")
  print("Model response: \n",test_finetuned_model(ds_row['prompt']))
  print("Actual result: \n",ds_row['chosen'])

In [51]:
for i in [42,68]:
  print("\n\n")
  test_on_dataset(ds['validation'][i])




####################
Model response: 
 user
Optimize a Python program that squares an array of integers so that it consumes the least possible memory.
assistant
def square_array(arr):
    return arr ** 2

result = square_array([1, 2, 3, 4, 5])
print(result) # Output: 120

# Optimized version
def square_array(arr):
    return arr ** 2

result = square_array([1, 2, 3, 4, 5])
print(result) # Output
Actual result: 
 def square_array(a):
    res = [x ** 2 for x in a]
    return res



####################
Model response: 
 user
Write a function for converting a series of hex values into a decimal integer in Python.
assistant
def hex_to_decimal(hex_str):
    return int(hex_str, 16)

hex_str = "0123456789ABCDEF"
decimal_int = hex_to_decimal(hex_str)
print(decimal_int) # Output: 15

# Output: 15

# Input: 0123456789ABC
Actual result: 
 def convert_hexvals_to_decimal(hexvals):
    # Split hexvals on whitespace
    hexvals_list = hexvals.split()
    # Reverse hexvals list
    hexvals_list.rev

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.